# State tomography

In [ ]:
import numpy as np
from pyquil.api import get_qc
from pyquil import Program
from pyquil.gates import I, X, RX, RY, H,CZ

from forest_qcvv.tomography import \
    generate_state_tomography_experiment, \
    acquire_tomography_data, \
    linear_inv_state_estimate

## Set the shots, the qubits of interest, the quantum device

In [ ]:
qubits = [0,1]
qvm = get_qc("9q-square-qvm", as_qvm=True, noisy=False)

## Prepare the state of interest and generate tomography programs

In [ ]:
state_prep = Program([H(q) for q in qubits])
state_prep.inst(CZ(0,1))
print(state_prep)

In [ ]:
exp_desc = generate_state_tomography_experiment(state_prep)

In [ ]:
print(exp_desc.program)

In [ ]:
[str(exp_desc.out_ops[idx]) for idx in list(range(0,(2**len(qubits))**2-1))]

# Acquire data

In [ ]:
exp_data = acquire_tomography_data(exp_desc, qvm, 0.001)

In [ ]:
exp_data

In [ ]:
est = linear_inv_state_estimate(exp_data)

In [ ]:
type(est)

In [ ]:
np.round(est.estimate.state_point_est,2)

In [ ]:
np.trace(np.matmul(est.estimate.state_point_est, est.estimate.state_point_est))

In [ ]:
from pyquil.api import WavefunctionSimulator
wfs = WavefunctionSimulator()
wf = wfs.wavefunction(state_prep)
yo = wf.amplitudes
target = np.outer(yo, yo)
print(target)

## Linear inversion estimate

In [ ]:
rho_est = linear_inv_state_estimate(exp_data).estimate.state_point_est

print(np.round(rho_est, 4))
print('Purity = ', np.trace(np.matmul(rho_est, rho_est)))

In [ ]:
type(linear_inv_state_estimate(exp_data))

## Maximum Liklihood Estimate (MLE) via diluted iterative method

In [ ]:
from forest_qcvv.tomography import iterative_mle_state_estimate

In [ ]:
est_mle,status = iterative_mle_state_estimate(exp_data, dilution=0.5)
rho = est_mle.estimate.state_point_est
print(np.around(rho, decimals=4))
print('Purity = ', np.trace(rho @ rho))

In [ ]:
type(est_mle)

## MLE with Max Entropy constraint

In [ ]:
est_max_ent, stat = iterative_mle_state_estimate(exp_data, dilution=0.5,entropy_penalty=0.005)
rho = est_max_ent.estimate.state_point_est
print(np.around(rho, decimals=4))
print('Purity = ', np.trace(rho @ rho))

## MLE with Hedging parameter

In [ ]:
est_hedging, stat = iterative_mle_state_estimate(exp_data, dilution=0.5, beta=.61)
rho = est_hedging.estimate.state_point_est
print(np.around(rho, decimals=4))
print('Purity = ', np.trace(rho @ rho))

## Project an unphysical state to the closest physical state

In [ ]:
from forest_qcvv.tomography import project_density_matrix

In [ ]:
rho_un = np.array([[1.0, 0], [0, -0.75]])
phys = project_density_matrix(rho_un)
print(phys)

In [ ]:
# Test the wizard method. Example from fig 1 of maximum likelihood minimum effort 
# https://doi.org/10.1103/PhysRevLett.108.070502

eigs = np.diag(np.array(list(reversed([3.0/5, 1.0/2, 7.0/20, 1.0/10, -11.0/20]))))
phys = project_density_matrix(eigs)
np.allclose(phys,np.diag([0, 0, 1.0/5, 7.0/20, 9.0/20]))

# Lightweight Bootstrap for functionals of the state

In [ ]:
import forest_qcvv.distance_measures as dm
from forest_qcvv.tomography import estimate_variance

In [ ]:
def my_mle_estimator(data):
    return  iterative_mle_state_estimate(data, dilution=0.5, entropy_penalty=0.0, beta=0.0)[0]

**Purity**

In [ ]:
mle_est = estimate_variance(exp_data, my_mle_estimator, dm.purity, n_resamples=40, project_to_physical=True)
lin_inv_est = estimate_variance(exp_data, linear_inv_state_estimate, dm.purity, n_resamples=40, project_to_physical=True)
print(mle_est)
print(lin_inv_est)

**Fidelity**

In [ ]:
mle_est = estimate_variance(exp_data, my_mle_estimator, dm.fidelity, target_state=target, n_resamples=40, project_to_physical=True)
lin_inv_est = estimate_variance(exp_data, linear_inv_state_estimate, dm.fidelity, target_state=target, n_resamples=40, project_to_physical=True)
print(mle_est)
print(lin_inv_est)

In [ ]:
dm.fidelity((rho_est+rho_est.conj().T)/2, target)